# Corrigé Projet Maison n° 3 - Session 5

In [ ]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation ci-dessus).

In [ ]:
def df_names_us():
    dfs = [(pd
            .read_csv(f"names/yob{year}.txt", names=["name", "gender", "births"])
            .assign(year=year))
           for year in range(1880, 2022)]
    
    df = (pd.concat(dfs, ignore_index=True)
          .loc[:, ["year", "name", "gender", "births"]]
         )
    
    return df

In [ ]:
%%time
df_us = df_names_us()
df_us.shape

In [ ]:
# head
df_us.head()

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [ ]:
# with converters
def df_names_fr():
    # genres
    mapping = {1:"M", 2:"F"}
    # read_csv
    df = pd.read_csv("nat2021_csv.zip",
                      sep=";",
                      header=0,
                      names=["gender", "name", "year", "births"],
                      converters={"name": str.title, "gender": mapping.get})
    # clean
    df = (df
          .loc[lambda df_: (df_.name.str.len() > 1) & (df_.year != "XXXX") & ~(df_.name.str.startswith("_"))]
          #.query("""(name.str.len() > 1) and (year != "XXXX") and not name.str.startswith("_")""", engine='python')
          .assign(year=lambda df_: df_.year.astype(int))
          .loc[:, ["year", "name", "gender", "births"]]
          .sort_values(["year", "gender", "births", "name"], ascending=[True, True, False, True])
          .reset_index(drop=True)
         )
    
    return df

In [ ]:
%%time
df_fr = df_names_fr()
df_fr.shape

In [ ]:
# with NA values
def df_names_fr():
    # genres
    mapping = {1:"M", 2:"F"}
    # read_csv
    df = pd.read_csv("nat2021_csv.zip",
                      sep=";",
                      header=0,
                      names=["gender", "name", "year", "births"],
                      na_values={"year":"XXXX", "name":"_PRENOMS_RARES"},
                      keep_default_na=False)
    # clean
    df = (df
          .dropna()
          .loc[lambda df_: df_.name.str.len() > 1]
          .assign(year=lambda df_: df_.year.astype(int),
                  name=lambda df_: df_.name.str.title(),
                  gender=lambda df_: df_.gender.map(mapping)
                 )
          .loc[:, ["year", "name", "gender", "births"]]
          .sort_values(["year", "gender", "births", "name"], ascending=[True, True, False, True])
          .reset_index(drop=True)
         )
    
    return df

In [ ]:
%%time
df_fr = df_names_fr()
df_fr.shape

In [ ]:
# head
df_fr.head()

In [ ]:
# prénom NA
df_fr.loc[df_fr['name']=='Na']

In [ ]:
# read_csv default
df = pd.read_csv("nat2021_csv.zip",
                  sep=';',
                  header=0,
                  names=["gender", "name", "year", "births"]
                )

df["name"].isnull().sum()

In [ ]:
# read_csv mode string sans NA par défaut
df = pd.read_csv("nat2021_csv.zip",
                  sep=';',
                  header=0,
                  names=["gender", "name", "year", "births"],
                  dtype=str,
                  na_values="",
                  keep_default_na=False
                )

df["name"].isnull().sum()

In [ ]:
# use functools.partial
from functools import partial

load_raw_csv = partial(pd.read_csv, dtype=str, na_values="", keep_default_na=False)

In [ ]:
# usage of load_raw_csv
(load_raw_csv("nat2021_csv.zip", sep=";")
 .isna()
 .sum()
)

In [ ]:
# capitalize
"JEAN-MARIE".capitalize()

In [ ]:
# title
"JEAN-MARIE".title()

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format '%d/%m/%Y') et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [ ]:
def df_taux_change(devises):
    df = pd.read_csv("Webstat_Export.csv",
                     sep=";",
                     na_values='-',
                     decimal=',',
                     skiprows=range(1, 6),  # le skiprows permet à l'option "decimal" de fonctionner
                     converters={0: lambda x: pd.to_datetime(x, format='%d/%m/%Y', errors='ignore')})

    # extraction des codes iso3 des monnaies
    iso3 = pd.Series(df.columns.tolist()).str.extract('\(([A-Z]{3})\)$', expand=False)
    iso3.iloc[0] = 'Date'
    
    df = (df.rename(columns=dict(zip(df.columns, iso3)))
          .loc[:, ['Date'] + devises]
          .dropna()
          .set_index('Date')
          .sort_index()
         )
    
    return df

In [ ]:
# exemple
df_tx = df_taux_change(["CHF", "GBP", "USD"])
df_tx.shape

In [ ]:
# head
df_tx.head()

In [ ]:
# index
df_tx.index

### Tests

In [ ]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [ ]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)
    
run_tests()

### Session 5 - Un peu de Data Science...

**Question n° 1**

Pourquoi le `value_counts()` du "gender" donne-t-il un tel écart entre "F" et "M" ?

In [ ]:
# value_counts du gender US

df_us["gender"].value_counts()

In [ ]:
# value_counts du gender FR

df_fr["gender"].value_counts()

In [ ]:
# pct_change

df_us["gender"].value_counts().pct_change()

In [ ]:
# pct_change

df_fr["gender"].value_counts().pct_change()

**Question n° 2**

Pourquoi le `value_counts()̀  du "name" donne-t-il ce résultat ?

In [ ]:
# value_counts du gender US

df_us["name"].value_counts().head(16)

In [ ]:
# value_counts du gender FR

df_fr["name"].value_counts().head(16)

**Question n° 3**

Pourquoi le `value_counts()` du "year" donne-t-il ce résultat ?

In [ ]:
# value_counts du gender US

df_us["year"].value_counts().head(16)

In [ ]:
# value_counts du gender FR

df_fr["year"].value_counts().head(16)

**Exercice n° 1**

Donnez le prénom qui a été le plus donné lors d'une année.

**Exercice n° 2**

Donnez la liste des prénoms qui contiennent dans l'ordre a, e, i, o et u (US) ou bien a, e, i et o (FR).

### Méthodes de reshaping (1)

#### pivot_table

La méthode `pivot_table()` prend en argument :
- index : colonne(s) dont les valeurs vont servir d'index à la table pivot
- columns : colonne(s) dont les valeurs vont servir de colonnes à la table pivot
- values : valeurs qui doivent être agrégées selon les modalités de la colonne passée en "index" et de la colonne passée en "columns"
- aggfunc : fonction d'aggrégation des values, par défaut 'mean', 'median', 'min', 'max', 'count', 'sum', 'nunique', et n'importe quelle lambda ou liste de fonctions
- fill_value : valeur en cas d'absence des modalités croisées 

On obtient `NaN` s'il n'y a pas d'occurence croisée.

Excel = tableau croisé dynamique.

In [ ]:
# exemple
df = pd.DataFrame([{'A': 1,'B': 1, 'C': 1},
                   {'A': 1,'B': 1, 'C': 2},
                   {'A': 1,'B': 2, 'C': -1},
                   {'A': 2,'B': 1, 'C': 4},
                   {'A': 2,'B': 1, 'C': 5},
                  ])

df

In [ ]:
# exemple
tab = df.pivot_table(values='C',
                     index='A',
                     columns='B')
tab

In [ ]:
type(tab)

In [ ]:
tab.index

In [ ]:
tab.columns

In [ ]:
# exemple
df.pivot_table(values='C',
              index='A',
              columns='B',
              aggfunc=tuple)

In [ ]:
# exemple
df.pivot_table(values='C',
              index='A',
              columns='B',
              aggfunc='sum')

**Exercice n° 3**

Calculez une table pivot avec le nombre total de naissances par année et par genre.

**Exercice n° 4**

Calculez une table pivot avec la diversité des prénoms (nombre de prénoms distincts) par année et par genre.

#### crosstab

`crosstab()` est une fonction de reshaping qui prend 2 Series partageant le même index (ou 2 colonnes d'un DataFrame) en argument et produit le décompte croisé des occurrences.

On obtient 0 s'il n'y a pas d'occurence croisée.

In [ ]:
# exemple
pd.crosstab(df['A'], df['B'])

In [ ]:
# initial
df_us['initial'] = df_us['name'].str[0].str.upper()
# terminal
df_us['terminal'] = df_us['name'].str[-1].str.upper()

df_us.head()

In [ ]:
# crosstab
pd.crosstab(df_us['initial'], df_us['terminal'])

In [ ]:
# pivot_table
df_us.pivot_table(values='name',
                  index='initial',
                  columns='terminal',
                  aggfunc='count',
                  fill_value=0)

In [ ]:
# Z x Z
df_us.loc[(df_us['initial'] == 'Z') & (df_us['terminal'] == 'Z')]

In [ ]:
# Z x Z
df_us.loc[df_us['name'].str.startswith('Z') & df_us['name'].str.endswith('z')]